### Importing non-mrcnn libraries and setting up directories

In [ ]:
import os
import sys
import random
import numpy as np
import skimage.io as skio
import matplotlib
import matplotlib.pyplot as plt
import copy
from skimage import measure
%matplotlib inline

# Root directory of the project
ROOT_DIR = os.path.abspath("/home/rave/tana-crunch/waves/deepimagery/data/raw/wv2/")

EIGHTCHANNEL_DIR = os.path.join(ROOT_DIR, 'eightchannels')
TRAIN_DIR = os.path.join(ROOT_DIR, 'train')
TEST_DIR = os.path.join(ROOT_DIR, 'test')
WV2_DIR = os.path.join(ROOT_DIR, 'gridded_wv2')
LABELS_DIR = os.path.join(ROOT_DIR, 'gridded_wv2_labels')
CONNECTED_COMP_DIR = os.path.join(ROOT_DIR, 'connected_comp_labels')
OPENED_LABELS_DIR = os.path.join(ROOT_DIR, 'opened_labels')
try:
    os.mkdir(OPENED_LABELS_DIR)
    os.mkdir(CONNECTED_COMP_DIR)
    os.mkdir(EIGHTCHANNEL_DIR)
    os.mkdir(MODEL_DIR)
    os.mkdir(TRAIN_DIR)
    os.mkdir(TEST_DIR)
    os.mkdir(WV2_DIR)
    os.mkdir(LABELS_DIR)
except:
    FileExistsError

os.chdir(ROOT_DIR)

### Stacking on/off season pairs into single 8 channel tif files and moving them to 8channel directory. I've rmeoved partial grids, which cuts out over 200 samples, yikes!

In [ ]:
def load_merge_wv2(image_id, source_dir):
    """Load the specified wv2 os/gs image pairs and return a [H,W,8] 
    Numpy array. Channels are ordered [B, G, R, NIR, B, G, R, NIR], OS 
    first.
    """
    # Load image
    os_path = source_dir+'/'+image_id+'_MS_OS.tif'
    gs_path = source_dir+'/'+image_id+'_MS_GS.tif'
    os_image = skio.imread(os_path)
    gs_image = skio.imread(gs_path)
    # If has more than 4 bands, select correct bands 
    # will need to provide image config in future
    # to programmaticaly use correct band mappings
    if os_image.shape[-1] != 4:
        os_image = np.dstack((os_image[:,:,1:3],os_image[:,:,4],os_image[:,:,6]))
    if gs_image.shape[-1] != 4:
        gs_image = np.dstack((gs_image[:,:,1:3],gs_image[:,:,4],gs_image[:,:,6]))
    stacked_image = np.dstack((os_image, gs_image))
    stacked_image_path = EIGHTCHANNEL_DIR +'/'+ image_id + '_OSGS_ms.tif'
    assert stacked_image.ndim == 3
    if -1.7e+308 not in stacked_image:
        #removes partial grids
        skio.imsave(stacked_image_path,stacked_image, plugin='tifffile')
    else:
        #might try later, keeps partial grids and sets nodata to 0
        # but no examples i've seen use partial images for training like this commented code
        #stacked_image[stacked_image==-1.7e+308]=0
        #skio.imsave(stacked_image_path,stacked_image, plugin='tifffile')
# all files, including ones we don't care about
file_ids_all = next(os.walk(WV2_DIR))[2]
# all multispectral on and off season tifs
image_ids_all = [image_id for image_id in file_ids_all if 'MS' in image_id]
#check for duplicates
assert len(image_ids_all) == len(set(image_ids_all))

image_ids_gs = [image_id for image_id in image_ids_all if 'GS' in image_id]
image_ids_os = [image_id for image_id in image_ids_all if 'OS' in image_id]

#check for equality
assert len(image_ids_os) == len(image_ids_gs)

image_ids_short = [image_id[0:9] for image_id in image_ids_gs]

for imid in image_ids_short:
    load_merge_wv2(imid, WV2_DIR)

In [ ]:
def show_arr(image_id):
    """Load the specified wv2 os/gs image pairs and return a [H,W,8] 
    Numpy array. Channels are ordered [B, G, R, NIR, B, G, R, NIR], OS 
    first.
    """

    stacked_image_path = EIGHTCHANNEL_DIR +'/'+ image_id + '_OSGS_ms.tif'

    arr = skio.imread(stacked_image_path, plugin='tifffile')
    
    plt.figure()
    arr[arr==-1.7e+308]=0
    skio.imshow(arr[:,:,0])
    plt.title(str(np.min(arr))+' '+stacked_image_path)
    return arr
# for imid in image_ids_short:
#     arr = show_arr(imid)

In [ ]:
image_list = next(os.walk(EIGHTCHANNEL_DIR))[2]
def move_img_to_folder(filename):
    '''Moves a file with identifier pattern ZA0165086_MS_GS.tif to a 
    folder path ZA0165086/image/ZA0165086_MS_GS.tif
    Also creates a masks folder at ZA0165086/masks'''
    
    folder_name = os.path.join(TRAIN_DIR,filename[:9])
    os.mkdir(folder_name)
    new_path = os.path.join(folder_name, 'image')
    mask_path = os.path.join(folder_name, 'masks')
    os.mkdir(new_path)
    file_path = os.path.join(EIGHTCHANNEL_DIR,filename)
    os.rename(file_path, os.path.join(new_path, filename))
    os.mkdir(mask_path)
    
for img in image_list:
    move_img_to_folder(img)
print('done moving images')

### Erosion/Dilation to get rid of really small partial fields created due to edge effects

In [ ]:
label_list = next(os.walk(LABELS_DIR))[2]

In [ ]:
from skimage import morphology as skim
for name in label_list:
    arr = skio.imread(os.path.join(LABELS_DIR,name))
    arr[arr == -1.7e+308]=0
    label_name = name[0:15]+'.tif'
    opened_path = os.path.join(OPENED_LABELS_DIR,name)
    kernel = np.ones((5,5))
    arr = skim.binary_opening(arr, kernel)
    arr=1*arr
    assert arr.ndim == 2
    skio.imsave(opened_path, 1*arr)

### Connected Components on wv2 labels, move them to connected comps folder

Does keeping no data from partial grid coverage lead to too much edge effects?
Should the arrays be opened before saved as seperate masks? The degree to which they are is a tuning param. 

In [ ]:
label_list = next(os.walk(OPENED_LABELS_DIR))[2]

In [ ]:
for name in label_list:
    arr = skio.imread(os.path.join(OPENED_LABELS_DIR,name))
    blob_labels = measure.label(arr, background=0)
    print(blob_labels.shape, 'blob')
    blob_vals = np.unique(blob_labels)
    for blob_val in blob_vals[blob_vals!=0]:
        labels_copy = blob_labels.copy()
        labels_copy[blob_labels!=blob_val] = 0
        labels_copy[blob_labels==blob_val] = 1
        label_name = name[0:15]+str(blob_val)+'.tif'
        label_path = os.path.join(CONNECTED_COMP_DIR,label_name)
        print(labels_copy.shape, 'copy')
        assert labels_copy.ndim == 2
        skio.imsave(label_path, labels_copy)

In [ ]:
def move_mask_to_folder(filename):
    '''Moves a mask with identifier pattern ZA0165086_label_1.tif to a 
    folder path ZA0165086/mask/ZA0165086_label_01.tif. Need to run 
    connected components first.
    '''
    if os.path.isdir(os.path.join(TRAIN_DIR,filename[:9])):
        folder_path = os.path.join(TRAIN_DIR,filename[:9])
        mask_path = os.path.join(folder_path, 'masks')
        file_path = os.path.join(CONNECTED_COMP_DIR,filename)
        os.rename(file_path, os.path.join(mask_path, filename))
    
mask_list = next(os.walk(CONNECTED_COMP_DIR))[2]
for mask in mask_list:
    move_mask_to_folder(mask)
print('Done moving masks')

### After masks are moved, run np.zeros_like to create empty mask from image for folders only where there is no mask. Keras expects mask for each image even if there are no instances. There are over 300 out of 794 grids without any fields, could lead to big class imbalance

In [ ]:
import pandas as pd
id_list = next(os.walk(TRAIN_DIR))[1]
no_field_list = []
for fid in id_list:
    mask_folder = os.path.join('train',fid, 'masks')
    if not os.listdir(mask_folder): 
        no_field_list.append(mask_folder)
no_field_frame = pd.DataFrame(no_field_list)
no_field_frame.to_csv('no_field_list.csv')

In [ ]:
import shutil
for fid in id_list:
    mask_folder = os.path.join(ROOT_DIR,'train',fid, 'masks')
    im_folder = os.path.join(ROOT_DIR,'train',fid, 'image')
    if not os.listdir(mask_folder): 
        im_path = os.path.join(im_folder, os.listdir(im_folder)[0])
        arr = skio.imread(im_path)
        mask = np.zeros_like(arr[:,:,0])
        print(mask.shape)
        assert mask.ndim == 2
        skio.imsave(os.path.join(mask_folder, fid + '_label_0.tif'),mask)
        shutil.rmtree(os.path.join('train',fid)) #removing no field images

Calculating mean pixel across all grids for each channel, ignoring nans

In [ ]:
def get_arr_channel_mean(channel):
    means = []
    for i, fid in enumerate(id_list):
        im_folder = os.path.join('train',fid, 'image')
        im_path = os.path.join(im_folder, os.listdir(im_folder)[0])
        arr = skio.imread(im_path)
        arr[arr==-1.7e+308]=np.nan
        means.append(np.nanmean(arr[:,:,channel]))
    return np.mean(means)
print(get_arr_channel_mean(0))
print(get_arr_channel_mean(1))
print(get_arr_channel_mean(2))
print(get_arr_channel_mean(3))
print(get_arr_channel_mean(4))
print(get_arr_channel_mean(5))
print(get_arr_channel_mean(6))
print(get_arr_channel_mean(7))


Plotting connected components

In [ ]:
testdir = os.path.join(ROOT_DIR,'train/ZA0953012/masks')
labs = next(os.walk(testdir))[2]
os.chdir(testdir)
for path in labs:
    arr = skio.imread(path)
    all_labels = measure.label(arr)
    plt.figure(figsize=(9, 3.5))
    plt.subplot(131)
    plt.imshow(arr, cmap='gray')
    plt.title(path)
    plt.tight_layout()
    plt.show()

In [ ]:
os.chdir(LABELS_DIR)
for path in label_list[0:50]:
    arr = skio.imread(path)
    arr[arr == -1.7e+308]=0
    all_labels = measure.label(arr)
    blob_labels = measure.label(arr, background=0)
    plt.figure(figsize=(9, 3.5))
    plt.subplot(131)
    plt.imshow(arr, cmap='gray')
    plt.axis('off')
    plt.subplot(132)
    plt.imshow(all_labels, cmap='spectral')
    plt.axis('off')
    plt.subplot(133)
    plt.imshow(blob_labels, cmap='spectral')
    plt.axis('off')
    plt.title(path)
    plt.tight_layout()
    plt.show()

Does keeping no data from partial grid coverage lead to too much edge effects?

In [ ]:
arr[arr == -1.7e+308]=0

In [ ]:
np.unique(arr)